# Cloud seeding (Problem 10 in Wasserman 7.4)

In 1975, an experiment was conducted to see if cloud seeding produced rainfall. 26 clouds were seeded with silver nitrate and 26 were not. The decision to seed or not to seed was made at random. We would like to check whether rainfall happens significantly more often in the seeded population.

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os

Let us load the data on the precipitations from the clouds. Note that in this data file, the first few lines are filled up with supplementary information. Therefore we need to explicitly tell Pandas where the data starts.

In [4]:
cwd = os.getcwd()

#If on MAC, this will likely work
datadir = '/'.join(cwd.split('/')[0:-1]) + '/Data/'
#If on window's machine, explicitly put in data dir
#datadir = 

# Telling Pandas that the data starts on line 23 in the file
df = pd.read_table(datadir + 'clouds.dat', sep = '\s+', header=23)

## Analysis based on the mean

Let us regard these data as samples from two distributions $F_1$ and $F_2$ with means

$$\mu_1=\int x dF_1(x)\qquad \mu_2 = \int xdF_2(x).$$

The plug in estimates are

$$\hat\mu_1 = \bar X_{n,1} = 164.58\qquad \hat\mu_2 = \bar{X}_{n,2}= 441.98$$

as one can see for example:

In [5]:
df.describe()

,Unseeded_Clouds,Seeded_Clouds
count,26.000000,26.000000
mean,164.588462,441.984615
std,278.426404,650.787171
min,1.000000,4.100000
25%,24.825000,98.125000
50%,44.200000,221.600000
75%,159.200000,406.025000
max,1202.600000,2745.600000


Alternatively, one can also write:

In [6]:
# Both forms do the same
means=np.array([df.Unseeded_Clouds.mean(), df['Seeded_Clouds'].mean()])
means

array([ 164.58846154,  441.98461538])

Clearly the seeded cloud produce larger mean rainfall. But is this statistically significant?

Recall that the standard error of the sample mean $\hat{\mu}$ is

$$se(\hat\mu)=\sqrt{\mathrm{V}\left(\frac{1}{n}\sum_{i=1}^nX_i\right)} = \sqrt{\left(\frac{1}{n}\sum_{i=1}^n\mathrm{V}(X_i)\right)}=\sqrt{\frac{n\sigma^2}{n^2}}=\frac{\sigma}{\sqrt{n}},$$

which we estimate by

$$\hat{se}(\hat\mu)=\frac{\hat{\sigma}}{\sqrt{n}},\qquad \hat{\sigma}=\sqrt{\frac{1}{n}\sum_{i=1}^n(X_i-\bar X_n)^2}$$

Let us compute these estimated standard errors in our case, first by computing the $\hat\sigma$ values:

In [7]:
n = len(df.index)
shat_unseeded = np.sqrt(((df.Unseeded_Clouds - df.Unseeded_Clouds.mean()) ** 2).mean())
shat_seeded = np.sqrt(((df.Seeded_Clouds - df.Seeded_Clouds.mean()) ** 2).mean())

sigmas = np.array([shat_unseeded, shat_seeded])
sigmas

array([ 273.01955175,  638.14932403])

Note that these values are exactly $\sqrt{\frac{n}{n-1}}$ times the values shown in *df.describe()* above. This is because

$$std^2 = S^2 = \frac{1}{n-1}\sum_{i=1}^n(X_i-\bar X_n)^2.$$

Indeed:

In [60]:
np.sqrt(n/(n-1)) * sigmas

array([ 278.42640439,  650.78717117])

(Of course, we could have just used $S^2$ as an estimate to begin with, but then how would I have made this point?) Next let's compute the estimated standard errors:

In [65]:
std_errors = np.sqrt(1/n) * sigmas

This allows us to write down approximate $95\%$ normal based confidence intervals $\hat\mu\pm 2\hat{se}(\hat\mu)$:

In [81]:
lower, upper = [], []

for i in range(2):
    lower.append(means[i] - 2 * std_errors[i])
    upper.append(means[i] + 2 * std_errors[i])

print("The two confidence intervals for $\mu_1$ and $\mu_2$ are")
print("({:0.2f},{:0.2f})) and ({:0.2f},{:0.2f})".format(lower[0],upper[0], lower[1], upper[1]))

The two confidence intervals for $\mu_1$ and $\mu_2$ are
(57.50,271.68)) and (191.68,692.29)


The two intervals overlap, so this is not conclusive.

Consider instead the functional $\theta = \mu_2 - \mu_1$. The plug-in estimate is

$$\hat\theta = \hat\mu_2-\hat\mu_1 = 277.4.$$

Indeed:

In [86]:
mean_diff = means[1]-means[0]

The standard error of $\hat\theta$ is

$$se = \sqrt{\mathrm{V}\left(\hat\mu_2-\hat\mu_1\right)}=\sqrt{\mathrm{V}\left(\hat\mu_2\right)+\mathrm{V}\left(\hat\mu_1\right)}=\sqrt{\left(se(\hat\mu_1)\right)^2+\left(se(\hat\mu_2)\right)^2}$$,

where we used the independence of the two cloud populations. This latter formula can be estimated as

$$\hat{se} = \sqrt{\left(\hat{se}(\hat\mu_1)\right)^2+\left(\hat{se}(\hat\mu_2)\right)^2}= 136.12 $$.

Indeed:

In [84]:
se_diff = np.sqrt(std_errors[0] ** 2 + std_errors[1] ** 2)
se_diff

136.12412822344402

Therefore, an approximate $95\%$ confidence interval for $\theta$ is:

In [88]:
print("({:0.2f},{:0.2f})".format(mean_diff-se_diff,mean_diff+se_diff))

(141.27,413.52)


Note that this interval is on the positive halfline suggesting that seeding indeed improves rainfall.

## Analysis based on the median using bootstrap

In [102]:
median_diff = df.Seeded_Clouds.median() - df.Unseeded_Clouds.median()

In [95]:
B = 1000 # Bootstrap sample size
N = len(df.index) # Original sample size
median_boot = []

for i in range(B):
    unseeded = np.array([df.Unseeded_Clouds.iloc[np.random.randint(0,N)] for j in range(N)])
    seeded = np.array([df.Seeded_Clouds.iloc[np.random.randint(0,N)] for j in range(N)])
    median_boot.append(np.median(seeded)-np.median(unseeded))

In [98]:
se_median_boot = np.std(median_boot)

The normal based confidence interval is therefore:

In [103]:
[median_diff-2*se_median_boot, median_diff + 2* se_median_boot]

[55.561655395355913, 299.23834460464406]

While the percentile based is:

In [111]:
[np.percentile(median_boot, 0.025, interpolation = 'higher'), np.percentile(median_boot, 0.975)]

[-4.2999999999999972, 36.121412499999998]

In [107]:
median_boot

[169.05000000000001,
 143.80000000000001,
 249.44999999999999,
 204.55000000000001,
 186.5,
 286.80000000000001,
 180.5,
 190.84999999999999,
 206.19999999999999,
 212.15000000000001,
 169.05000000000001,
 209.69999999999999,
 61.099999999999994,
 166.80000000000001,
 71.700000000000003,
 77.900000000000006,
 130.09999999999999,
 187.69999999999999,
 191.25,
 167.60000000000002,
 211.5,
 154.44999999999999,
 167.0,
 188.80000000000001,
 100.8,
 241.89999999999998,
 157.5,
 158.54999999999998,
 130.0,
 81.75,
 159.94999999999999,
 236.05000000000001,
 190.84999999999999,
 117.39999999999999,
 134.44999999999999,
 230.5,
 140.69999999999999,
 271.15000000000003,
 126.89999999999998,
 61.100000000000001,
 163.04999999999998,
 71.700000000000003,
 91.499999999999972,
 200.19999999999999,
 191.95000000000002,
 239.64999999999998,
 227.39999999999998,
 8.6999999999999886,
 69.75,
 239.64999999999998,
 216.79999999999998,
 194.45000000000002,
 77.049999999999997,
 38.599999999999994,
 186.5,
